In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
from selenium_stealth import stealth # 셀레니움 스텔스 기능

# 예외 처리
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

import time
import pandas as pd
import numpy as np
import os
import random

In [2]:
# smartDF 만드는 함수
def makeLinkDF(storelink, storename):
    smartDF = pd.DataFrame([storelink, storename])
    smartDF.index=['링크','상점']
    smartDF = smartDF.T
    return smartDF

# 기존 파일영역(saveDF)에 추가(newDF)해줌.
def concatFile(saveDF, newDF): 
    saveDF=pd.read_excel('./Data/output.xlsx')
    saveDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 불러오면 'Unnamed: 0' 제거해줘야함.
    updateDF=pd.concat([saveDF, newDF])
    return updateDF

# 스크롤을 내리는 함수 (페이지 끝까지)
def scroll_to_end(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # 페이지가 로드되는 동안 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# 링크 DF 만드는 함수 (상품, 해당 링크)
def makeLinks(name_list, link_list):
    dataDF = pd.DataFrame([name_list, link_list])
    dataDF.index=['상품','링크']
    dataDF = dataDF.T
    return dataDF

# smartDF 파일 업데이트하기
def updateLink(smartDF, filename):
    baseDF=pd.read_excel(filename)
    smartDF=pd.concat([baseDF, smartDF])
    smartDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 'Unnamed: 0' 제거해줘야함.
    smartDF.to_excel(filename)
    return smartDF

In [3]:
# 사용할 파일들
file = './rawfile2.xlsx'
textfile = './textfile2.xlsx'

# 엑셀 파일 읽기
fileDF = pd.read_excel(file)

# 전체 담을 텍스트 리스트 만들기
text_list = []

for link in fileDF['링크']:
    agent_option = webdriver.ChromeOptions()
    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)
    driver = webdriver.Chrome(options=agent_option)
    time.sleep(1)

    driver.get(link)
    driver.implicitly_wait(5)
    page_num = driver.find_elements(By.CLASS_NAME, "UWN4IvaQza._nlog_click")
    button = len(page_num)
    driver.quit()
    time.sleep(1)

    ### 페이지 루프 돌면서 텍스트 가져오기
    for i in range(1, button+1):
        try:
            agent_option = webdriver.ChromeOptions()
            user_agent_string = 'Mozilla/5.0'
            agent_option.add_argument('user-agent=' + user_agent_string)
            driver = webdriver.Chrome(options=agent_option)

            target = link + f'?filterMyComment=false&page={i}'
            driver.get(target)
            time.sleep(1)

            # 페이지별 텍스트 추출 리스트
            text_in_page = []
            
            # HTML 가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # 필요한 데이터 추출
            texts = [tag.text.strip() for tag in soup.find_all('div', class_='_38rS4qj1Bp')]
            text_in_page.extend(texts)
            text_list.append(text_in_page)
            driver.quit()
            time.sleep(1)
            
        # 오류 발생 시 예외 처리 후 넘어감.
        except Exception as e:
            print(f'예외 오류 발생 코드: {e}')
            continue
            

    # 결과 저장
    textDF = pd.DataFrame({'텍스트': text_list})
    textDF.to_excel(textfile, index=False)